In [1]:
## Mount Google Drive Data
try:
    from google.colab import drive
    drive.mount("/content/drive")
except:
    print("Mounting Failed.")

Mounted at /content/drive


In [ ]:
# ## Unzip files
# import tarfile
# tarfile_pth = '/content/drive/MyDrive/Capstone_code/Task02_Heart.tar'
# output_pth = '/content/drive/MyDrive/Capstone_code/Data'
# with tarfile.open(tarfile_pth, 'r') as tar:
#     tar.extractall(path=output_pth)

In [2]:
## Prepare Mamba environment
!pip install packaging
!pip install causal-conv1d
!pip install mamba-ssm
!pip install timm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 6.6 MB/s eta 0:00:00
  Created wheel for causal-conv1d: filename=causal_conv1d-1.4.0-cp310-cp310-linux_x86_64.whl size=104867883 sha256=b5e7cf7e964b5e99275d97ba1e1b0ee4e3073f4593743ba1f1c6aa394a3008cc
  Stored in directory: /root/.cache/pip/wheels/e3/dd/4c/205f24e151736bd22f5980738dd10a19af6f093b6f4dcab006
Successfully built causal-conv1d
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.0 MB/s eta 0:00:00
  Created wheel for mamba-ssm: filename=mamba_ssm-2.2.2-cp310-cp310-linux_x86_64.whl size=323988104 sha256=6b082468a6abb6f6bc50c99263f17c6c7f5a2e8f6b275ed7998b81fb25279229
  Stored in directory: /root/.cache/pip/wheels/57/7c/90/9f963468ecc3791e36e388f9e7b4a4e1e3f90fbb340055aa4d
Successfully built mamba-ssm


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import sys
import argparse

sys.path.append('/content/drive/MyDrive/capstone_code/utils')
sys.path.append('/content/drive/MyDrive/capstone_code/models')

from trainer import Trainer
from dataset import get_data, ImageDataset
from visualization import visualize_label
from visualization import visualize_prediction
from evaluation import evaluation

import UNet, TransUnet, MambaUnet #, MambaUnet

/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:164: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, xz, conv1d_weight, conv1d_bias, x_proj_weight, delta_proj_weight,
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/selective_scan_interface.py:240: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, dout):
/usr/local/lib/python3.10/dist-packages/mamba_ssm/ops/triton/layer_norm.py:986: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `to

In [17]:
# 跑这个
parser = argparse.ArgumentParser(description='Training')
# Define path
parser.add_argument("--data_path", type=str, default="/content/drive/MyDrive/capstone_code/Data/Task06_Lung")
parser.add_argument("--model_path", type=str, default="/content/drive/MyDrive/capstone_code/checkpoints")
# Whether to continue training
parser.add_argument("--ct", type=bool, default=False)

# parser.add_argument("--model_name", type=str, default="unet")
# parser.add_argument("--model_name", type=str, default="transunet")
parser.add_argument("--model_name", type=str, default="mambaunet")
parser.add_argument("--num_classes", type=int, default=2)
parser.add_argument("--n_skip", type=int, default=3)
parser.add_argument("--num_epochs", type=int, default=200)
# Define learning rate
parser.add_argument("--learning_rate", type=int, default=0.001)
# Define batch size
parser.add_argument("--train_batch", type=int, default=16)
parser.add_argument("--val_batch", type=int, default=16)
parser.add_argument("--test_batch", type=int, default=1)
args = parser.parse_args([])

In [6]:
import numpy as np
# Use only labeled data
# all_image, all_label = get_data(args.data_path, args.num_classes)
all_image = np.load('/content/drive/MyDrive/capstone_code/all_image.npy')
all_label = np.load('/content/drive/MyDrive/capstone_code/all_label.npy')

In [8]:

# Training 80%
# Validation 10%
# Testing 10%
train_data, temp_data, train_label, temp_label = train_test_split(all_image, all_label, test_size=0.2, random_state=42)
val_data, test_data, val_label, test_label = train_test_split(temp_data, temp_label, test_size=0.5, random_state=42)
del all_image
del all_label

In [9]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(2602, 224, 224)
(325, 224, 224)
(326, 224, 224)


In [10]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(10)
])

In [11]:
train_dataset = ImageDataset(train_data, train_label, transform=transform)
val_dataset = ImageDataset(val_data, val_label)
test_dataset = ImageDataset(test_data, test_label)

train_dataloader = DataLoader(train_dataset, batch_size=args.train_batch, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=args.val_batch, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=args.test_batch, shuffle=False)

In [ ]:
#然后跑这个
if args.model_name in ["unet", "transunet", "mambaunet"]:
    if args.model_name == "unet":
        model = UNet.UNet(args.num_classes)
    elif args.model_name == "transunet":
        model = TransUnet.TransUnet(args.num_classes, args.n_skip)
    else:
        model = MambaUnet.MambaUnet(args.num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.num_epochs, eta_min=0.00001)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, min_lr=1e-6)

    trainer = Trainer(args)
    trainer.train(model, optimizer, scheduler, train_dataloader, val_dataloader)
else:
    print("Wrong model type!")

Start Training mambaunet...
No checkpoint found to load.

EPOCH 1 of 200

training loss: 0.2586473464234475,
validation loss: 0.25090425709883374

EPOCH 2 of 200

training loss: 0.2141891590382424,
validation loss: 0.20242661308674587

EPOCH 3 of 200

training loss: 0.1676021668991428,
validation loss: 0.1660855064789454

EPOCH 4 of 200

training loss: 0.15256453120361077,
validation loss: 0.13568531455738203

EPOCH 5 of 200

training loss: 0.14717504162729883,
validation loss: 0.16307330167009718

EPOCH 6 of 200

training loss: 0.13376800087423413,
validation loss: 0.1272057458048775

EPOCH 7 of 200

training loss: 0.11988276368909818,
validation loss: 0.11425892121735073

EPOCH 8 of 200

training loss: 0.1179205998092707,
validation loss: 0.08334213158204443

EPOCH 9 of 200

training loss: 0.11490845453976853,
validation loss: 0.1236128592420192

EPOCH 10 of 200

training loss: 0.10719009289986517,
validation loss: 0.08398306334302538

EPOCH 11 of 200

training loss: 0.11387499531536

In [ ]:
if args.model_name in ["unet", "transunet", "mambaunet"]:
    if args.model_name == "unet":
        model = UNet.UNet(args.num_classes)
        df_unet = evaluation(args, model, test_dataloader)
    elif args.model_name == "transunet":
        model = TransUnet.TransUnet(args.num_classes, args.n_skip)
        df_trans = evaluation(args, model, test_dataloader)
    else:
        model = MambaUnet.MambaUnet(args.num_classes)
        df_mamba = evaluation(args, model, test_dataloader)

else:
    print("Wrong model type!")

In [ ]:
import os
import pandas as pd
excel_path = os.path.join(args.model_path, "network_comparison_10_22.xlsx")
with pd.ExcelWriter(excel_path) as writer:
    df_unet.to_excel(writer, sheet_name="U-Net", index=False)
    df_trans.to_excel(writer, sheet_name="TransUNet", index=False)
    df_mamba.to_excel(writer, sheet_name="MambaUnet", index=False)

In [ ]:
import os
os._exit(00)

In [ ]:
# import matplotlib.pyplot as plt
# idx = 90
# sample = test_dataset[idx]
# sample_image = sample["data"].squeeze()
# sample_label = sample["label"]
# model = MambaUnet.MambaUnet(args.num_classes)
# plt.imshow(visualize_prediction(args, sample_image, model))
# plt.axis("off")
# plt.show()